## Chapter 11: Principles of Feature Learning

# 11.10 K-Fold Cross-Validation

You can toggle the code on and off in this presentation via the button below.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

- With ensembling <u>**human interpretability**</u> is typically lost as the final model is an average of many potentially very different nonlinearities. 

- K-fold cross-validation is often applied when interpretability of a final model is important.

- Instead of averaging a set of cross-validated models over many splits of the data (ensembling), with K-fold cross-validation we choose a single model that has minimum *average validation error* over all splits of the data.

- This produces a potentially less accurate final model, but one that is significantly simpler and hence more easily understood by humans.  

In [1]:
## This code cell will not be shown in the HTML version of this notebook
# imports from custom library
import sys
sys.path.append('../../')
from mlrefined_libraries import math_optimization_library as optlib
from mlrefined_libraries import nonlinear_superlearn_library as nonlib
from mlrefined_libraries import basics_library 

# demos for this notebook
regress_plotter = nonlib.nonlinear_regression_demos_multiple_panels
classif_plotter = nonlib.nonlinear_classification_visualizer_multiple_panels
static_plotter = optlib.static_plotter.Visualizer()
basic_runner = nonlib.basic_runner
classif_plotter_crossval = nonlib.crossval_classification_visualizer
datapath = '../../mlrefined_datasets/nonlinear_superlearn_datasets/'

# import autograd functionality to bulid function's properly for optimizers
import autograd.numpy as np

# import timer
from datetime import datetime 
import copy
import math

# this is needed to compensate for %matplotlib notebook's tendancy to blow up images when plotted inline
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

%load_ext autoreload
%autoreload 2

## The K-fold cross-validation procedure

To further simplify the final outcome of this procedure instead of using completely random training-validation splits, as done with ensembling, we split the data randomly into a set of $K$ non-overlapping pieces. This is depicted visually in [Figure 11.52](#figure-11-52), where the original data is represented as the entire circular mass (top left panel) is split into $K=3$ non-overlapping sets (bottom row).  We then cycle through $K$ training-validation splits of the data that consist of $K-1$ of these pieces as training, with the final portion as validation, which allows for each point in the dataset to belong to a validation set precisely one time.  Each such split is referred to as a *fold*, of which there are $K$ in total, hence the name 'K-folds.'  On each fold we cross-validate the *same set of models* and record the validation score of each.  Afterwards we choose the single best model that produced the lowest *average validation error*.  Once this is done the minimum average validation model is re-trained over the entire dataset to provide a final tuned predictor of the data.  

---

<a id='figure-11-52'></a>
<figure>
<p>
  <img src= '../../mlrefined_images/nonlinear_superlearn_images/Figure_11_52.png' width="70%"  alt=""/>
</p>
<figcaption> <em> 
Schematic illustration of K-fold cross-validation for
$K=3$.  The original data (top
left) is split into $K$ non-overlapping sets or folds. In each instance we keep a different
portion of the split data as validation while merging the remaining $K-1$
pieces as training.
</em>
</figcaption>
</figure>

---

Since no models are combined / averaged together with K-folds, it can very easily produce less accurate models (in terms of *testing error* - see Section 11.7) for general learning problems when compared to ensembling.  However, when human interpret-ability of a model overshadows the needs for exceptional performance, K-folds produces a stronger performing model than a single cross-validated model that can still be understood by human beings.  This is somewhat analogous to the story of feature selection detailed in Sections [11.5](https://jermwatt.github.io/machine_learning_refined/notes/11_Feature_learning/11_5_Boosting.html) and [11.6](https://jermwatt.github.io/machine_learning_refined/notes/11_Feature_learning/11_6_Regularization.html), where human interpret-ability is the guiding motivator (and not simply model effectiveness).  In fact the notion of feature selection and K-folds cross-validation indeed intersect in certain applications, one of which we will see in the Examples below.

---

#### <span style="color:#a50e3e;">Example.</span>   Galileo's gravity experiment

In this Example we use K-fold cross-validation on the Galileo dataset (see Example 10.2 in the book for details) using polynomial models of degree one through six. 

---

<a id='figure-11-53'></a>
<figure>
<p>
  <img src= '../../mlrefined_images/nonlinear_superlearn_images/Figure_11_53.png' width="80%"  alt=""/>
</p>
<figcaption> <em> 
(small panels) Six cross-validated models, each trained on all but one point from the dataset (this sometimes referred to as leave-one-out cross-validation). Here the validation portion of each fold (i.e., a single data point) is highlighted in yellow. (large panel) The model with lowest average validation error is a quadratic. 
</em>
</figcaption>
</figure>

---

##   K-fold cross-validation and high dimensional linear modeling

Suppose for a moment we have a high capacity model which enables several kinds of overfitting behavior for a nonlinear regression dataset, with each overfitting instance of the model provided by different settings of the linear combination weights of the model (suppose any internal parameters of its features are fixed).  We illustrate such a scenario in the left panel of [Figure 11.54](#figure-11-54), where two settings of such a model provide two distinct overfitting predictors for a generic nonlinear regression dataset.  As we learned in [Section 10.2](https://jermwatt.github.io/machine_learning_refined/notes/10_Nonlinear_intro/10_2_Regression.html), any *nonlinear model* in the original space of a regression dataset corresponds to a \emph{linear model} in the transformed feature space (i.e., the space where each individual input axis is given by one of the chosen nonlinear feature).  Since our model easily overfits the original data, in the transformed feature space our data lies along a *linear subspace* that can be perfectly fit using many different hyperplanes.  Indeed the two nonlinear overfitting models shown in the left panel of the Figure  correspond one-to-one with the two linear fits in the transformed feature space - here illustrated symbolically in the right panel of the Figure (in reality we could not visualize this space, as it would likely be too high dimensional).  In other words, a severely overfitting high capacity \emph{nonlinear model} in the original regression space is severely overfitting high capacity *linear model* in the transformed feature space feature.  This fact holds regardless of the problem type.

---

<a id='figure-11-54'></a>
<figure>
<p>
  <img src= '../../mlrefined_images/nonlinear_superlearn_images/Figure_11_54.png' width="90%"  alt=""/>
</p>
<figcaption> <em> For small datasets that have very high input dimension, even a linear model has extremely high capacity and can easily overfit.
(left panel) Two instances of overfitting by a high capacity model to a nonlinear regression dataset.  (right panel) These two models - as viewed in the feature transformed space - are linear.
</em>
</figcaption>
</figure>

---

- For small datasets that have very high input dimension, even a linear model has extremely high capacity and can easily overfit.

- In such scenarios, the use of more complicated nonlinear models is automatically ruled out.

- In order to properly tune the parameters of a high capacity (linear) model we often turn to regularization to block capacity.

- Given the small amount of data at play, K-fold cross-validation is commonly employed to determine the proper regularization parameter value and ultimately the parameters of the linear model.

This scenario often provides an interesting point of intersection with the notion of *feature selection via regularization* detailed in [Section 9.7](https://jermwatt.github.io/machine_learning_refined/notes/9_Feature_engineer_select/9_7_Regularization.html).  Employing the $\ell_1$ regularizer we can block the capacity of our high capacity linear model *while simultaneously* selecting important input features, making human interpret-ability possible.

---

#### <span style="color:#a50e3e;">Example.</span>   Genome-wide association studies

- Genome-wide association studies (GWAS) aim at understanding the connections between tens of thousands of genetic markers, taken from across the human genome of several subjects, with diseases like high blood pressure, diabetes, etc.

- These studies typically produce small datasets of high dimensional (input) genetic information taken from a sample of patients with a given affliction and a control group of non-afflicted.

- Thus regularization based cross-validation is a useful tool for learning meaningful (linear) models for such data.

---

<a id='figure-11-55'></a>
<figure>
<p>
  <img src= '../../mlrefined_images/nonlinear_superlearn_images/Figure_11_55.png' width="80%"  alt=""/>
</p>
<figcaption> <em> 
Conceptual illustration of a genome-wide association study wherein a quantitative biological trait (e.g., blood pressure or glucose level) is to be associated with specific genomic locations.
</em>
</figcaption>
</figure>

- Using a feature selecting regularizer like the $\ell_1$ norm can help researchers identify the handful of genes critical to affliction, which can both improve our understanding of disease and perhaps provoke development of gene-targeted therapies. 


---